In [ ]:
import feedparser
from logging import getLogger
import logging
from pprint import pprint as print
from bs4 import BeautifulSoup
from src.util import init_logger, md5hash_6
import yaml

In [ ]:
md5hash_6("https://www.ezindie.com/feed/rss.xml")

In [ ]:
init_logger()
logger = getLogger()

In [ ]:
cfg = feedparser.parse("https://www.reddit.com/r/China_irl/.rs?")
cfg.get("bozo_exception")

In [ ]:
# 读取pickle
import pickle
with open("resource/cache.pkl", "rb") as f:
    feeds = pickle.load(f)
print(feeds)

In [ ]:
from openai import Model
import os
model = Model()
res = model.list(os.environ.get("OPENAI_API_KEY"))
res